In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT,GPSConfig,ResultConfig
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import json
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

In [2]:
#load network data and graph
network = Network("../osmnx_example/beijing/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 157738 edges 393216
[2025-02-22 20:06:03.469] [info] [network.cpp:72] Read network from file ../osmnx_example/beijing/edges.shp
[2025-02-22 20:06:07.219] [info] [network.cpp:172] Number of edges 393216 nodes 157738
[2025-02-22 20:06:07.219] [info] [network.cpp:173] Field index: id 18 source 0 target 1
[2025-02-22 20:06:07.514] [info] [network.cpp:176] Read network done.
[2025-02-22 20:06:07.515] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-02-22 20:06:07.566] [info] [network_graph.cpp:30] Graph nodes 157738 edges 393216
[2025-02-22 20:06:07.566] [info] [network_graph.cpp:31] Construct graph from network edges end


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [ ]:
from fmm import UBODTGenAlgorithm
ubodt_gen = UBODTGenAlgorithm(network,graph)
status = ubodt_gen.generate_ubodt("../osmnx_example/beijing/ubodt.txt", 4, binary=False, use_omp=True)
print(status)

[2025-02-21 12:06:36.625] [info] [ubodt_gen_algorithm.cpp:76] Start to generate UBODT with delta 4
[2025-02-21 12:06:36.625] [info] [ubodt_gen_algorithm.cpp:77] Output format csv
Status: success
Time takes 0.82 seconds

[2025-02-21 12:06:36.708] [info] [ubodt_gen_algorithm.cpp:105] Progress 64 / 649
[2025-02-21 12:06:36.787] [info] [ubodt_gen_algorithm.cpp:105] Progress 128 / 649
[2025-02-21 12:06:36.866] [info] [ubodt_gen_algorithm.cpp:105] Progress 192 / 649
[2025-02-21 12:06:36.951] [info] [ubodt_gen_algorithm.cpp:105] Progress 256 / 649
[2025-02-21 12:06:37.046] [info] [ubodt_gen_algorithm.cpp:105] Progress 320 / 649
[2025-02-21 12:06:37.125] [info] [ubodt_gen_algorithm.cpp:105] Progress 384 / 649
[2025-02-21 12:06:37.199] [info] [ubodt_gen_algorithm.cpp:105] Progress 448 / 649
[2025-02-21 12:06:37.278] [info] [ubodt_gen_algorithm.cpp:105] Progress 512 / 649
[2025-02-21 12:06:37.370] [info] [ubodt_gen_algorithm.cpp:105] Progress 576 / 649
[2025-02-21 12:06:37.441] [info] [ubodt_gen

### FMM model creation and map matching

match single trajectory

In [ ]:
wkt = "LINESTRING(116.320147 39.984071, 116.320122 39.98414, 116.320114 39.984167, 116.319654 39.98497, 116.319714 39.984874, 116.319728 39.98487)"
ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

result = model.match_wkt(wkt,fmm_config)
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

[2025-02-22 20:26:54.276] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/beijing/ubodt.txt
[2025-02-22 20:26:55.319] [info] [ubodt.cpp:236] Read rows 1000000
[2025-02-22 20:26:56.199] [info] [ubodt.cpp:236] Read rows 2000000
[2025-02-22 20:26:57.248] [info] [ubodt.cpp:236] Read rows 3000000
[2025-02-22 20:26:58.105] [info] [ubodt.cpp:236] Read rows 4000000
[2025-02-22 20:26:58.920] [info] [ubodt.cpp:236] Read rows 5000000
[2025-02-22 20:26:59.673] [info] [ubodt.cpp:236] Read rows 6000000
[2025-02-22 20:27:00.467] [info] [ubodt.cpp:236] Read rows 7000000
[2025-02-22 20:27:01.267] [info] [ubodt.cpp:236] Read rows 8000000
[2025-02-22 20:27:02.062] [info] [ubodt.cpp:236] Read rows 9000000
[2025-02-22 20:27:03.140] [info] [ubodt.cpp:236] Read rows 10000000
[2025-02-22 20:27:04.005] [info] [ubodt.cpp:236] Read rows 11000000
[2025-02-22 20:27:04.807] [info] [ubodt.cpp:236] Read rows 12000000
[2025-02-22 20:27:05.609] [info] [ubodt.cpp:236] Read rows 13000000
[202

In [11]:
import folium
from shapely.wkt import loads

original_geom = loads(wkt)
matched_geom = loads(result.mgeom.export_wkt())

# Create map centered at the first point
m = folium.Map(location=[original_geom.coords[0][1], original_geom.coords[0][0]], zoom_start=16)

# Plot original trajectory (red)
folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords], color="red", weight=3, opacity=0.8, tooltip="Original GPS Trajectory").add_to(m)

# Plot matched trajectory (blue)
folium.PolyLine([(lat, lon) for lon, lat in matched_geom.coords], color="blue", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

# Add markers for key points
for lon, lat in original_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red").add_to(m)

for lon, lat in matched_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="blue", fill=True, fill_color="blue").add_to(m)

# Display map (if running in a Jupyter Notebook)
m

match multiple trajectories

In [ ]:
import pandas as pd
from shapely.geometry import Point, LineString
from io import StringIO

# Read CSV data into a DataFrame
data = "../osmnx_example/beijing/trajectory.csv"

# Open the file directly (no need to use StringIO unless you have the data as a string)
df = pd.read_csv(data, delimiter=';')

# Extract longitude and latitude
coordinates = list(zip(df['x'], df['y']))

# Create a list of Points
points = [Point(lon, lat) for lon, lat in coordinates]

# Create a LineString object from the Points
line = LineString(points)

# Print the resulting LineString
print(line)

import geopandas as gpd

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[line])

# Save the GeoDataFrame as a GeoJSON file
gdf.to_file("trajectory_line.geojson", driver="GeoJSON")

LINESTRING (116.320147 39.984071, 116.320122 39.98414, 116.320114 39.984167, 116.319654 39.98497, 116.319714 39.984874, 116.319728 39.98487, 116.319703 39.984821, 116.319717 39.984787, 116.319735 39.98483, 116.319711 39.984911, 116.31972 39.984917, 116.319724 39.984944, 116.319697 39.985, 116.319672 39.985063, 116.319682 39.985088, 116.319663 39.985136, 116.319626 39.985199, 116.31964 39.98521, 116.31964 39.985171, 116.319666 39.985123, 116.319668 39.985112, 116.319668 39.985112, 116.319666 39.985107, 116.319718 39.985076, 116.319739 39.985074, 116.319777 39.985062, 116.319847 39.985052, 116.319934 39.985054, 116.320001 39.985073, 116.320077 39.985073, 116.320141 39.98507, 116.320144 39.985043, 116.3202 39.985019, 116.320223 39.984965, 116.32025 39.984934, 116.320311 39.984906, 116.320289 39.984914, 116.320309 39.984863, 116.320309 39.984846, 116.320361 39.984821, 116.320361 39.984821, 116.320356 39.984819, 116.320416 39.984733, 116.320519 39.984684, 116.320618 39.984679, 116.320734 39

In [5]:
ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

input_config = GPSConfig()
input_config.file = "../osmnx_example/beijing/trajectory.csv"
input_config.id = "id"

result_config = ResultConfig()
result_config.file = "../osmnx_example/beijing/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

matched_df = pd.read_csv("../osmnx_example/beijing/mr.txt", delimiter=";")
print(matched_df.head())


[2025-02-22 20:15:16.798] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/beijing/ubodt.txt
[2025-02-22 20:15:17.928] [info] [ubodt.cpp:236] Read rows 1000000
[2025-02-22 20:15:18.751] [info] [ubodt.cpp:236] Read rows 2000000
[2025-02-22 20:15:19.684] [info] [ubodt.cpp:236] Read rows 3000000
[2025-02-22 20:15:20.590] [info] [ubodt.cpp:236] Read rows 4000000
[2025-02-22 20:15:21.477] [info] [ubodt.cpp:236] Read rows 5000000
[2025-02-22 20:15:22.389] [info] [ubodt.cpp:236] Read rows 6000000
[2025-02-22 20:15:23.313] [info] [ubodt.cpp:236] Read rows 7000000
[2025-02-22 20:15:24.209] [info] [ubodt.cpp:236] Read rows 8000000
[2025-02-22 20:15:25.052] [info] [ubodt.cpp:236] Read rows 9000000
[2025-02-22 20:15:25.977] [info] [ubodt.cpp:236] Read rows 10000000
[2025-02-22 20:15:26.864] [info] [ubodt.cpp:236] Read rows 11000000
[2025-02-22 20:15:27.662] [info] [ubodt.cpp:236] Read rows 12000000
[2025-02-22 20:15:28.666] [info] [ubodt.cpp:236] Read rows 13000000
[202

In [ ]:
# Load matched and original data
matched_df = pd.read_csv("../osmnx_example/stockholm/mr.txt", delimiter=";")
original_df = pd.read_csv("../osmnx_example/stockholm/trips.csv", delimiter=";")

# Merge original and matched trajectories by 'id'
merged_df = pd.merge(matched_df[['id', 'mgeom']], original_df[['id', 'geom']], on='id')

# Initialize lists for GeoJSON
original_geojson_list = []
matched_geojson_list = []

for _, row in merged_df.iterrows():
    # Convert original and matched geometry to GeoJSON
    original_geom = loads(row["geom"])  # 'geom' is the column for original trajectories
    matched_geom = loads(row["mgeom"])  # 'mgeom' is the column for matched trajectories
    
    original_geojson_list.append({
        "type": "Feature",
        "geometry": mapping(original_geom),
        "properties": {"id": row["id"], "type": "Original"}
    })
    
    matched_geojson_list.append({
        "type": "Feature",
        "geometry": mapping(matched_geom),
        "properties": {"id": row["id"], "type": "Matched"}
    })

# Create GeoJSON FeatureCollection for both original and matched
geojson_data = {"type": "FeatureCollection", "features": original_geojson_list + matched_geojson_list}

# Save to file (optional)
with open("merged_trajectories.geojson", "w") as f:
    json.dump(geojson_data, f)

# Initialize the map
m = Map(center=(59.34, 18.06), zoom=13)

# Add both original and matched layers with different colors
original_layer = GeoJSON(name="Original Trajectories", data={"type": "FeatureCollection", "features": original_geojson_list},
                         style={"color": "blue", "weight": 3})
matched_layer = GeoJSON(name="Matched Trajectories", data={"type": "FeatureCollection", "features": matched_geojson_list},
                        style={"color": "orange", "weight": 3})

m.add_layer(original_layer)
m.add_layer(matched_layer)

# Display the map
m


Map(center=[59.34, 18.06], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Map Matching after obfuscation